In [1]:
#导入包
import torch
from torchvision import datasets,transforms
import torch.nn as nn
import torch.optim as optim

In [2]:
#加载数据集
train_data = datasets.MNIST(root='data',train=True,transform=transforms.ToTensor(),download=True)
test_data = datasets.MNIST(root='data',train=False,transform=transforms.ToTensor(),download=True)
#root是存放数据位置；transform代表数据处理方式使用Totensor表示直接转换为张量（高维数组）；doenload设置为True表示root里有数据就用没有就下载到root

#打印出张量大小
len(train_data)
len(test_data)


10000

数据很多，所有采用批处理方式，就是从数据集种每次取一小部分进行训练

In [4]:
#数据批处理
batch_size=100
train_loader=torch.utils.data.DataLoader(dataset=train_data,batch_size=batch_size,shuffle=True)
test_loader=torch.utils.data.DataLoader(dataset=test_data,batch_size=batch_size,shuffle=False)
#shuffle表示打乱数据

**dataset是负责管理数据，dataloader负责批量获取数据**

In [6]:
#定义MLP网络继承nn.Module
class MLP(nn.Module):
    #初始化方法
    #input_size输入数据的维度 即图片的大小28*28
    #hidden_size隐藏层的大小  
    #num_classes输出类别的数目 10
    def __init__(self,input_size,hidden_size,num_classes):
        #调用父类的初始化方法
        super(MLP,self).__init__()
        #定义第一个全连接层
        self.fc1=nn.Linear(input_size,hidden_size)
        #定义激活函数
        self.relu=nn.ReLU()
        #定义第二个全连接层
        self.fc2=nn.Linear(hidden_size,hidden_size)
        #定义第三个全连接层
        self.fc3=nn.Linear(hidden_size,num_classes)

    #定义forward函数
    #x输入的数据
    def forward(self,x):
        #第一层运算
        out=self.fc1(x)
        out=self.relu(out)
        #将上一步结果送入
        out=self.fc2(out)
        out=self.relu(out)
        out=self.fc3(out)
        return out

In [7]:
#定义模型
input_size=28*28
hidden_size=512
num_classes=10
model=MLP(input_size,hidden_size,num_classes)

#定义损失函数和优化器
criterion=nn.CrossEntropyLoss() #交叉熵损失函数
learning_rate=0.001
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

In [8]:
#模型训练
num_epochs=10 #训练轮数,十次训练每次都是batch_size
for epoch in range(num_epochs):
    for i,(image,labels) in enumerate(train_loader):
        image=image.reshape(-1,28*28)
        outputs=model(image)
        loss=criterion(outputs,labels)
        optimizer.zero_grad() #梯度清0
        loss.backward() #反向传播
        optimizer.step() #更新参数

        if (i+1)%300==0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}' )
        
        

Epoch [1/10], Step [300/600], Loss: 0.1298
Epoch [1/10], Step [600/600], Loss: 0.1854
Epoch [2/10], Step [300/600], Loss: 0.1410
Epoch [2/10], Step [600/600], Loss: 0.0491
Epoch [3/10], Step [300/600], Loss: 0.0881
Epoch [3/10], Step [600/600], Loss: 0.0471
Epoch [4/10], Step [300/600], Loss: 0.0620
Epoch [4/10], Step [600/600], Loss: 0.0525
Epoch [5/10], Step [300/600], Loss: 0.0059
Epoch [5/10], Step [600/600], Loss: 0.0188
Epoch [6/10], Step [300/600], Loss: 0.0061
Epoch [6/10], Step [600/600], Loss: 0.0197
Epoch [7/10], Step [300/600], Loss: 0.0175
Epoch [7/10], Step [600/600], Loss: 0.0266
Epoch [8/10], Step [300/600], Loss: 0.0519
Epoch [8/10], Step [600/600], Loss: 0.0085
Epoch [9/10], Step [300/600], Loss: 0.0054
Epoch [9/10], Step [600/600], Loss: 0.0013
Epoch [10/10], Step [300/600], Loss: 0.0074
Epoch [10/10], Step [600/600], Loss: 0.0283


In [9]:
#测试网络
with torch.no_grad(): #表示测试过程不用pytorch计算梯度
    correct=0
    total=0
    for images,labels in test_loader:
        images=images.reshape(-1,28*28)
        outputs=model(images)
        _,predicted=torch.max(outputs.data,1) #max是取最大值的索引
        total+=labels.size(0)
        correct+=(predicted==labels).sum().item()
    print(f'正确率： {100*correct/total} %')

正确率： 98.1 %


In [10]:
#保存网络
torch.save(model,"mnist_mlp_model.pkl")